# WaveNet-Lite Moonshot Kill Test
**Goal**: Test dilated causal CNN (WaveNet-lite) as completely different architecture from GRU.
No recurrence — gated residual blocks with exponential dilations, skip connections.
Receptive field = 1024 steps (covers full sequence).

**Kill gate**: mean val (3 seeds) > 0.2709.

**Hypothesis**: GRU's sequential bottleneck limits what it can learn.
CNN processes all positions in parallel with global receptive field.
Ring buffers enable online inference without recurrence.

**Pipeline**: Setup -> Train 3 seeds -> Evaluate -> Save

In [ ]:
# Cell 1: Mount Drive, download data from Kaggle, clone repo
import os, json, subprocess

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

# Clone repo
REPO = "/content/competition_package"
os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"

commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train WaveNet models (seeds 42, 43, 44)
import subprocess, sys, os
os.chdir("/content/competition_package")

SEEDS = [42, 43, 44]
CONFIG = "configs/wavenet_v1.yaml"

print(f"=== WAVENET-LITE MOONSHOT ({len(SEEDS)} seeds) ===")
print(f"Config: {CONFIG} (channels=64, skip=96, 10 blocks, dilations 1..512)")
print("=" * 60, flush=True)

for seed in SEEDS:
    ckpt = f"logs/wavenet_v1_seed{seed}.pt"
    if os.path.exists(ckpt):
        print(f"seed {seed}: checkpoint exists -- skip")
        continue
    print(f"\n{'='*60}")
    print(f"WAVENET seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    rc = proc.wait()
    if rc != 0:
        print(f"ERROR: seed {seed} failed with rc={rc}")

print(f"\nTraining complete!")

In [ ]:
# Cell 3: Evaluate — compare WaveNet vs vanilla GRU baseline
import os, torch
os.chdir("/content/competition_package")

SEEDS = [42, 43, 44]
BASELINE_VALS = {42: 0.2649, 43: 0.2737, 44: 0.2690}  # vanilla GRU baseline reference
GATE = 0.2709  # kill gate

print(f"{'Seed':<6} {'WaveNet':>12} {'GRU Base':>10} {'Delta':>10}")
print("-" * 45)

scores = []
for seed in SEEDS:
    pt = f"logs/wavenet_v1_seed{seed}.pt"
    if not os.path.exists(pt):
        print(f"s{seed}:  MISSING")
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "?")
    baseline = BASELINE_VALS.get(seed, 0)
    delta = score - baseline
    scores.append(score)
    print(f"s{seed:<5} {score:>12.4f} {baseline:>10.4f} {delta:>+10.4f}  (ep {epoch})")

if scores:
    mean_wn = sum(scores) / len(scores)
    mean_base = sum(BASELINE_VALS[s] for s in SEEDS) / len(SEEDS)
    mean_delta = mean_wn - mean_base
    print(f"\nMean WaveNet:     {mean_wn:.4f}")
    print(f"Mean GRU base:    {mean_base:.4f}")
    print(f"Mean delta:       {mean_delta:+.4f}")
    print(f"\nKill gate ({GATE:.4f}): {'PASS' if mean_wn > GATE else 'FAIL'}")
    if mean_wn > GATE:
        print("-> Scale to 10 seeds + submit!")
    elif mean_wn > 0.2695:
        print("-> Near miss. Train 5 more seeds to check.")
    else:
        print("-> KILL WaveNet.")

In [ ]:
# Cell 4: Save checkpoints to Drive
import os, torch, shutil
os.chdir("/content/competition_package")

SEEDS = [42, 43, 44]
DRIVE_DIR = "/content/drive/MyDrive/wunderfund"

for seed in SEEDS:
    pt = f"logs/wavenet_v1_seed{seed}.pt"
    if os.path.exists(pt):
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        slim = {
            "model_state_dict": ckpt["model_state_dict"],
            "config": ckpt.get("config", {}),
            "best_score": ckpt.get("best_score", None),
            "best_epoch": ckpt.get("best_epoch", None),
        }
        dst = f"{DRIVE_DIR}/wavenet_v1_seed{seed}.pt"
        torch.save(slim, dst)
        sz = os.path.getsize(dst) / 1e6
        print(f"Saved: wavenet_v1_seed{seed}.pt ({sz:.1f}MB)")

print("\nDone!")